# Sorting out basic design issues

## Final Design Version before Cleaning

In [18]:
import numpy as np
import random
from collections import Counter

class User:
    """
    This is a relatively simple class that models the user (i,e the one that clicks the ad linkk or not)
    The primary function is the user response to when an ad is shown (ie show_ad)
    """
    
    def __init__(self):
        # Initializes the probability with the random-uniform distribution
        self.probability = random.uniform(0, 1)

    def show_ad(self):
        # The show ad function is simple in that it returns a True / False value based on 
        # a random choice of the probability distribution
        return  np.random.choice((True, False), p = [self.probability, 1-self.probability])

class Auction:
    
    """
    The auction class sets the bidding rules and manages both the bidders and users
    It also determines the winning bid and the price (which is the second highest)
    Also, included is the interaction with the bidder to request and then notify the bidder of the outcome
    The user function models the user (i,e the one that clicks the ad linkk or not)
    
    Key Assumption: Given the max reward is $1, we will limit max bids to at most $5
    """
    
    def __init__(self, users, bidders):
        
        # Initializes the auction class with users, bidders, auction history
        # balances are tracked by bidder
        
        self.users = users
        self.bidders = bidders
        self.auction_history = []
        
        # To note, I have used id(bidder) instead of (bidder) as that is incomprehensible
        self.balances = {id(bidder): 0 for bidder in bidders} ########### NEW LINE
        
        #print("*"*100, '\n')
        #print('Beginning balances : ',self.balances, '\n','-'*100)
        
        
    def execute_round(self):
        
        ############  EXECUTE ROUND  #### This is the principal function orchestrator #####################
        ###################################################################################################
        
        
        all_invalid_bidders = any(x > -1000 for x in self.balances.values())
        
        
        #########  if the -ve balance of ALL bidders is < -1000, then break ###############################
        
        while not all_invalid_bidders:
            return "no bidders are qualifed as all balances are below -1000"
            break
        
        #########  Proceed with the auction is even 1 bidder has a balance > -1000 #########################
        ####################################################################################################
        
        else:
            # Initialize dictionary and lists for tracking bids
            bids_dict = {}
            bids_list = []
            bids_list_raw = []

            ########## STEP 1. SELECT USER (from the pool of users) ########################################

            chosen_user = random.randint(0, len(self.users)-1)
            #print('Selected User: ', id(chosen_user), '\n','-'*100) 

            ########### STEP 2. COLLECT BIDS (send to all bidders) #########################################
            
            for bidder in self.bidders:
                if self.balances[id(bidder)] > -1000:
                    bids_dict[bidder] = bidder.bid(chosen_user)
                    bidding_round = bidder.bidding_round

                    self.auction_history.append((id(bids_dict[bidder]), bids_dict[bidder])) #bidder
                    highest_bid = 0
                    winning_price = 0
                else:
                    print("Bidder ", id(bidder), "balance is less than -1000. Cannot continue bid" )
     #               #raise Exception("Bidder's balance is less than -1000. Cannot continue bid")
                    continue

            print("Current bidding Round: ", bidding_round, '\n','-'*100)

            ############ STEP 3. DETERMINE WINNING BID ####################################################
            for bidder, bid_value in bids_dict.items():

                bids_list_raw.append((bidder, bid_value))
                ## only needed to track a human readable bidder id
                bids_list.append((id(bidder), bid_value))

            ########### STEP 4. SORT THE Bids List based on price #########################################
            sorted_list = sorted(bids_list, key=lambda t:t[1])
            #print('Sorted bids list', sorted_list, '\n','-'*100 )

            #------------- for analysis only - creating a copy with id(bidder) ---------------------------
            sorted_list_raw = sorted(bids_list_raw, key=lambda t:t[1])
    #        print('sorted bids list RAW', sorted_list_raw, '\n','-'*100 )


            ########### STEP 5. Select second highest price ###############################################
            if len(sorted_list) > 1:
                winning_price = (sorted_list_raw)[-2][1]
                winning_bidder = (sorted_list_raw)[-1][0]
            else:
                winning_price = (sorted_list_raw)[0][-1]
                winning_bidder = (sorted_list_raw)[0][0]

    #        print('Winning Price: ', winning_price, '\n','-'*100)
    #        print('winning bidder', winning_bidder, '\n','-'*100)

            ########### STEP 6. Determine if there are more than 1 bidders with same price ################

            multiple_winning_bidders = [tup for tup in sorted_list_raw if tup[1] == winning_price]

            if len(multiple_winning_bidders)>1:
                #print('List of biders that submitted similar bids: ' , multiple_winning_bidders, '\n','-'*100)
                randindex = random.randint(0, len(multiple_winning_bidders)-1)
                winning_bidder = multiple_winning_bidders[randindex][0]
                #print('Winning bidder from list of similar priced bids: ', winning_bidder, '\n','-'*100)
            else: 
                pass
                #winning_bidder = multiple_winning_bidders[0][0]

            print('Winning Bidder: ', id(winning_bidder), '\n','-'*100)
            print('Winning Price: ', winning_price, '\n','-'*100)

            ############ STEP 7. Validate USER CLICK after SHOW AD ###########################################

            ad_result = self.users[chosen_user].show_ad()
            #print('User Clicked on Ad?: ', ad_result,'\n','-'*100)

            ############# STEP 8.NOTIFY BIDDER & UPDATE BALANCES #############################################

            for bidder in self.bidders:
                if bidder == winning_bidder and ad_result == True:
                    #print(id(bidder), "bidder == winning_bidder and ad_result == True:")
                    self.balances[id(bidder)] -= winning_price
                    self.balances[id(bidder)] += 1

                    bidder.notify(True, winning_price, ad_result)

                elif bidder == winning_bidder and ad_result == False:
                    #print(id(bidder), "bidder == winning_bidder and ad_result == False")
                    self.balances[id(bidder)] -= winning_price

                    bidder.notify(True, winning_price, ad_result)

                else:
                    #print(id(bidder), "Did not win bid - only notify")
                    bidder.notify(False, winning_price, None)
            #print('-'*100, '\n','balances at end of bidding round', self.balances,'\n','-'*100)
        
class Bidder:
    
    """
    This is the bidder class where we track bidding details, patterns and history. 
    In addition to the required functions, there are five other functions that have been 
    created to model bidding strategies -- listed below
    
    Note the acronyms user
    BWCY: Bidder Won, Clicked Yes
    BWCN: Bidder Won, Clicked No
    BLOS: Bidder Lost
    """
    
    def __init__(self, num_users, num_rounds):
            
    ############ Initialize Attributes for bid and history  ###########################################
    ###################################################################################################
        
        ########### initializing basic attributes ###########
        self.num_users = num_users
        self.num_rounds = num_rounds
        self.bidding_round = 0
        self.bid_price = 0

        ########### Basic tracking metrics - dictionaries ###########
        self.bid_participate = {i: 0 for i in range(num_users)} #whether bidder bids or not (USER: BID COUNT)
        self.bid_wins_no_click = {i: 0 for i in range(num_users)} # if bidder wins
        self.bid_win_user_clicks = {i: 0 for i in range(num_users)} # if user clicks
        self.bid_lost = {i: 0 for i in range(num_users)}

        ########### Historical transaction tracking for trend analysis ###########
        self.bid_win_click_no = []
        self.bid_win_click_yes = []
        self.bid_lost_list = []
        self.bid_360 = []
        self.user_id = 0
        
    ############ SUBMIT BID  ########### initiated by auction #########################################
    ###################################################################################################
    
    def bid(self, user_id):
        """
        Function to set teh default (aka base) bid price
        The bid price is called from a separate function that determines the bid strategy
        """

        self.bidding_round += 1
        self.user_id = user_id
        self.bid_price = self.bid_strategy(user_id)
        #print('bid price from STRATEGY', self.bid_price, '\n','-'*100 )
        return self.bid_price

    ############  NOTIFIED OF BID OUTCOME  ########### initiated by auction ###########################
    ###################################################################################################
    
    def notify(self, auction_winner, price, clicked):
        bid_y_n_won = 0
        self.auction_winner = auction_winner
        self.price = price
        self.clicked = clicked
        
        if auction_winner == True and clicked == True:
            self.bid_win_user_clicks[self.user_id] = (self.bid_win_user_clicks).get(self.user_id, 0)+1
            self.bid_win_click_yes.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            self.bid_360.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])

        elif auction_winner == True and clicked == False:
            self.bid_wins_no_click[self.user_id] = (self.bid_wins_no_click).get(self.user_id, 0)+1
            self.bid_win_click_no.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            self.bid_360.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
        
        else:
            self.bid_lost[self.user_id] = (self.bid_lost).get(self.user_id, 0)+1
            self.bid_lost_list.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            self.bid_360.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
    
    ############  BID STRATEGY - CONTROL FLOW  ########################################################
    ###################################################################################################
    
    def bid_strategy(self, user_id):

        """
        This function controls the strategy and changes this with what it learns from the bidding process
        There are two basic - default strategies that will be followed for the first 50 rounds which are
        (1) start slow and observe bidding behavior for first 10 rounds
        (2) go aggressive and match max price of past rounds and in crease base bid by 10%
        After the first 50 rounds, the strategy revolves through 4 models (A, B, C & D) aso detailed below
        """
        
        ########## SET THE BASE BID PRICE  ##### Revise this based on Strategy ############################
        #################### Ranges from 0 to $1 ##########################################################
        self.base_bid_price = random.randint(0, 10000)/10000 #random.uniform(0, 1)
        ###################################################################################################
        
        trx_master = [x for x in self.bid_360]
        strategy_list = ['A', 'B', 'C', 'D']
        strategy = random.choice(strategy_list)
        
        for item in trx_master:
        
        ########## Observe: Start Slow for the first 10 rounds  ###########################################
        ###################################################################################################
        
            if item[1] < 2:
                self.bid_price = 0.01
            
        ########## Next 40 rounds, match max price + 10%  #################################################
        ###################################################################################################
            
            elif item[1] > 2 and item[1] < 4:
                self.bid_price = max(self.bid_360, key=lambda x: x[6])[6]*1.10
                if self.bid_price > self.base_bid_price * 100:
                    self.bid_price = self.base_bid_price
        
        ########## Rounds 50+ Learn, Optimize, Mitigate Losses and Win  ###################################
        ###################################################################################################
        
            elif strategy == 'A':
                self.bid_price = self.strategy_a()
            
            elif strategy == 'B':
                self.bid_price = self.strategy_b()
                
            elif strategy == 'C':
                self.bid_price = self.strategy_c()
                
            elif strategy == 'D':
                self.bid_price = self.strategy_d()
            
            else:
                self.bid_price = self.base_bid_price
            
        
        return self.bid_price
        
    
    ############  STRATEGY A - Identify losses and counter  ###########################################
    ###################################################################################################
        
    def strategy_a(self):
            
        """
        Setting min bid above average difference of loss for prior 10 bids  was greater than 10%
        then adjust you next bid to in increase by that amount
        """

        # filter last 10 losses  
        a = [x for x in self.bid_360[-10:] if x[0] == 'BLOS' and (x[6]-x[5]) > (.1*x[5])]

        # calculated the average percentage loss (> 10%)
        avg_loss_10percent = ((sum(i[6] for i in a) - sum(i[5] for i in a))/(max(1, len(a))))

        #adjust bid price
        self.bid_price = self.base_bid_price + avg_loss_10percent
        if self.bid_price > self.base_bid_price * 5:
            self.bid_price = self.base_bid_price
        return self.bid_price

    ############  STRATEGY B - Exploit/Focus on top User   ############################################
    ###################################################################################################
    
    def strategy_b(self):
        self.user_id = bid_strategy.user_id
        
        """
        Focus on most successful user --- ie where bidder has worn atleast 5 times for same user
        For this user, double the base bid price to increase odds of winning again
        """
        
        a = [x for x in self.bid_360[-100:] if x[0] == 'BWCY']
        #print(a)
        b = (Counter([x[3] for x in a]))
        #print(b)
        # filter for success > 5 times
        c = { k: v for k, v in b.items() if v >= 5 }
        # get max user (most successful thus far)
#        d = max(c, key=c.get, default=0) # get user with max wins
        
#        if self.user_id ==  max(c, key=c.get, default=0):
        # self.bid_price = self.base_bid_price*2
        # return self.bid_price
        if self.user_id ==  max(c, key=c.get, default=0):
            self.bid_price = self.base_bid_price*2
            return self.bid_price
        
    
    ############  STRATEGY C - Identify Max leaders and counter   #####################################
    ###################################################################################################
    
    def strategy_c(self):
#        self.a = a
        
        """
        Find Max winning bid in past 10 bids and increase your base bid by 20%
        """

        a = [x for x in self.bid_360[-10:] if x[0] == 'BLOS']
        self.bid_price = max(a, key=lambda x: x[6])[6]*1.20
        if self.bid_price > self.base_bid_price * 5:
            self.bid_price = self.base_bid_price
        return self.bid_price
    
    
    ############  STRATEGY D - Conserve cash when <-500   #############################################
    ###################################################################################################
    
    def strategy_d(self):
            
        """
        Monitor if balance goes < -500 then scale back on the bids by 10% of the base price
        Conserve cash and wait for other random strategies A, B or C to try again
        """

        # sum all bids that were won, but user not clicked (ie BWCN)
        a = [x for x in self.bid_360 if x[0] == 'BWCN'] 

        # calculate cost of bids in BWCN
        b = (sum(i[6] for i in a))
        
        # sum of all bids where user clicked (ie BWCY)
        c = [x for x in self.bid_360 if x[0] == 'BWCY']
        # sum of total cost
        d = (sum(i[6] for i in c) )

        # earnings till date (ie $1 for every BWCY)
        e = len(c)
        f = e * 1

        current_balance = (b + d -f)
        
        if current_balance < -500:
            self.bid_price = 0.01
        return self.base_bid_price    
    

In [19]:
#b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
b1, b2, b3, b4, b5, b6, b7, b8, b9, b10 = Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(10)], [b1, b2, b3, b4, b5, b6, b7, b8, b9, b10])
#auction = Auction( [User(), User()], [b1, b2, b3])
auction.execute_round()

Current bidding Round:  1 
 ----------------------------------------------------------------------------------------------------
Winning Bidder:  140535501490592 
 ----------------------------------------------------------------------------------------------------
Winning Price:  0 
 ----------------------------------------------------------------------------------------------------


In [48]:
b1, b2 = Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(10)], [b1, b2])

#auction = Auction( [User(), User()], [b1, b2, b3])
auction.execute_round()

Selected User:  140598975851056 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0 
 ----------------------------------------------------------------------------------------------------
Current bidding Round:  1 
 ----------------------------------------------------------------------------------------------------
Sorted bids list [(140597919164928, 0), (140597919165360, 0)] 
 ----------------------------------------------------------------------------------------------------
Winning bidder from list of similar priced bids:  <__main__.Bidder object at 0x7fdf81072a00> 
 ----------------------------------------------------------------------------------------------------
Winning Bidder (FINAL):  140597919164928 
 --------------------------------------------------------------------------

In [17]:
bal = auction.balances
bal

{140659521303552: -16.520833936376356,
 140659521303648: -8.873428291112624,
 140659521305904: -36.623229772132774,
 140659521305184: -12.44856547729011,
 140659521305856: -23.55766476365462,
 140659521305952: -21.897973868307883,
 140659521305616: -41.871889760850074,
 140659521305664: -10.31757061292901,
 140659521302784: -28.829125208703946,
 140659521302592: -18.743580269615745}

In [20]:
[auction.execute_round() for i in range(100)]

Current bidding Round:  2 
 ----------------------------------------------------------------------------------------------------
Winning Bidder:  140535501490832 
 ----------------------------------------------------------------------------------------------------
Winning Price:  0.01 
 ----------------------------------------------------------------------------------------------------


NameError: name 'bid_strategy' is not defined

# Consolidate Metrics: All Bidders

## Overall

In [45]:
# Summary Counts
print('Bidding Rounds', b1.bidding_round)

BWCY = (len(b1.bid_win_click_yes), len(b2.bid_win_click_yes), \
      len(b3.bid_win_click_yes), len(b4.bid_win_click_yes),  len(b5.bid_win_click_yes), len(b6.bid_win_click_yes), \
      len(b7.bid_win_click_yes), len(b8.bid_win_click_yes), len(b9.bid_win_click_yes), len(b10.bid_win_click_yes))

sum(BWCY)
print('BWCY - Bid Win - Click YES: Count', sum(BWCY))

BWCN = (len(b1.bid_win_click_no),len(b2.bid_win_click_no),len(b3.bid_win_click_no)\
     ,len(b4.bid_win_click_no),len(b5.bid_win_click_no),len(b6.bid_win_click_no),len(b7.bid_win_click_no),\
    len(b8.bid_win_click_no),len(b9.bid_win_click_no),len(b10.bid_win_click_no))

sum(BWCN)

print('BWCN - Bid Win - Click NO: Count',sum(BWCN))

# GENERAL CLICK RATIO

print('Overall Click Ratio ', sum(BWCY) / (sum(BWCY)+sum(BWCN)) )

# BLOS = NOT RELEVANT
# print('BLOS - Bid LOST: Count',len(b1.bid_lost_list))

# print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 59
BWCN - Bid Win - Click NO: Count 42
Overall Click Ratio  0.5841584158415841


# Bid Refining - Filters

## Observe first 10 rounds

In [8]:
trx_master = [x for x in self.bid_360]

for item in trx_master:

    if item[1] < 10:
        self.bid_price = 0.01
    else:
        self.bid_price = base_bid_price

# trx_master = [x for x in b1.bid_360]

# for item in trx_master:
#     if item[1] < 10:
#         print('hey')
#     else:
#         print('hey no')

NameError: name 'self' is not defined

## Loss by > 10%

In [14]:
# lost the bid by a price > 10%

# filter last 10 bids for those you have lost 
a = [x for x in b1.bid_360[-10:] if x[0] == 'BLOS' and (x[6]-x[5]) > (.1*x[5])]
#print(a)

# calculated the average percentage loss (> 10%)
b = ((sum(i[6] for i in a) - sum(i[5] for i in a))/(max(1, len(a))))

print(b)

# then set bid_price = 

# [0]Category, [1]self.bidding_round, [2]id(self), [3]self.user_id, 
# [4]self.auction_winner, [5]#self.bid_price, [6]self.winning price, [7]self.clicked

0.03324855583759785


## Most successful user

In [41]:
# lost the bid by a price > 10%
user_id = 0
from collections import Counter

a = [x for x in b1.bid_360[-1000:] if x[0] == 'BWCY']
print(a)
b = (Counter([x[3] for x in a]))
print(b)
c = { k: v for k, v in b.items() if v >= 0 }
print(c)
d = max(c, key=c.get, default = 0) # get user with max wins
print(d)
#print(c)

if user_id == max(c, key=c.get, default = 0):
    print('this works')
#print(max(b, key=b.get))

# then set bid_price = 

# [0]Category, [1]self.bidding_round, [2]id(self), [3]self.user_id, 
# [4]self.auction_winner, [5]#self.bid_price, [6]self.winning price, [7]self.clicked

[['BWCY', 7, 140659256162288, 0, True, 2.5493499999999996, 2.32776, True], ['BWCY', 9, 140659256162288, 6, True, 2.4122508333333332, 2.0869, True], ['BWCY', 56, 140659256162288, 9, True, 4.064549395238772, 2.165417633104279, True], ['BWCY', 65, 140659256162288, 0, True, 2.1120271134422053, 0.8586, True], ['BWCY', 87, 140659256162288, 0, True, 2.640689710454851, 2.640689710454851, True], ['BWCY', 104, 140659256162288, 9, True, 2.0669332208909412, 0.961, True], ['BWCY', 110, 140659256162288, 6, True, 1.1020721763220986, 1.0598, True], ['BWCY', 120, 140659256162288, 0, True, 2.60900925457629, 2.081799598283614, True], ['BWCY', 155, 140659256162288, 3, True, 3.015625169076803, 1.9316, True], ['BWCY', 161, 140659256162288, 7, True, 2.5999199630038508, 1.6862, True], ['BWCY', 176, 140659256162288, 2, True, 1.5483605668183071, 0.6728, True], ['BWCY', 196, 140659256162288, 0, True, 2.5513547345882257, 2.1617260629653603, True], ['BWCY', 218, 140659256162288, 9, True, 3.8508744549847913, 0.918,

## Wait certain rounds, then accelerate

In [35]:
#a = [x for x in b1.bid_360[-100:] if x[0] == 'BWCN'] # < ]
a = [x for x in b1.bid_360[-100:] if 100 < x[1] < 200] #'BWCN'] # < ]
print(a)
     # then take default bid probability

[['BLOS', 101, 140568245737168, 6, False, 0.04786, 0.08659, None]]


## Match Max and go over 10%

---

In [51]:
b1.bid_360 # Category,self.bidding_round, id(self), self.user_id, self.auction_winner,
           #self.bid_price, self.winning price, self.clicked
#[x for x in b1.bid_360]    

a = [x for x in b1.bid_360[-10:] if x[0] == 'BLOS']

# a  = [x for x in b1.bid_360]
# a = [trx_master[-10:] if x[0] == 'BLOS']
#print(trx_master)
a

b = max(a, key=lambda x: x[6])[6]*1.20
print(b)

0.012


## Monitor balance and go convervative

In [62]:
# Monitor if balance goes < -500
# then scale back on the bids by 10% of the base price

# filter last 10 bids for those you have lost 
a = [x for x in b1.bid_360 if x[0] == 'BWCN'] 
#print(a)

# calculate current balance
b = (sum(i[6] for i in a))# - sum(i[5] for i in a))/(max(1, len(a))))
print(b)

c = [x for x in b1.bid_360 if x[0] == 'BWCY']
d = (sum(i[6] for i in c) )
print(d)
      
e = len(c)

f = e * 1


balance = (b + d -f)
print(balance)
# then set bid_price = 

1755.6575161955327
45.451559031805296
1797.109075227338


---

## By Bidder

### for b1

In [5]:
# Summary Counts
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b1.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b1.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b1.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 68
BWCY - Bid Win - Click YES: Count 6
BWCN - Bid Win - Click NO: Count 6
BLOS - Bid LOST: Count 73
Balances {140597641037712: -1137.3780524846661, 140597641037568: -2732.4172326878843, 140597641037904: -1900.750469521423, 140597641038000: -1062.6159572654012, 140597641038240: -1377.4532861741338, 140597641038048: -2864.717876085827, 140597641038192: -1360.1529239092456, 140597641038144: -1007.8288178352286, 140597641038528: -1582.690163221218, 140597641038624: -1835.7285647786969}


## for b2

In [19]:
# Summary Counts
print('Bidding Rounds', b2.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b2.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b2.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b2.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 80
BWCY - Bid Win - Click YES: Count 4
BWCN - Bid Win - Click NO: Count 7
BLOS - Bid LOST: Count 78
Balances {140609583389232: -3158.6218075228603, 140609583389472: -2036.9503334695264, 140609583388704: -1298.4005622498923, 140609583389088: -2422.3687593379536, 140609583389568: -1766.9887882813941, 140609583390528: -1248.6460170059604, 140609583390240: -1242.9306891310805, 140609583390048: -1558.7446566452027, 140609583389136: -1651.77405244511, 140608778463648: -1229.9684893382973}


## for b3

In [33]:
# Summary Counts
print('Bidding Rounds', b3.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b3.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b3.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b3.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 5
BWCN - Bid Win - Click NO: Count 10
BLOS - Bid LOST: Count 86
Balances {140588383663200: -1147.6122670914983, 140588388722480: -1234.6556190166716, 140588388723152: -683.9171783816711, 140588388722912: -1103.359507689096, 140588388721088: -1428.327987930117, 140588388722816: -1782.5122671418005, 140588388720848: -1279.3556726191655, 140588388722864: -1182.55272084212, 140588388721616: -454.9901072078393, 140588388720944: -1439.5127313027533}


---

# User Info

In [34]:
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Wins User Clicked', {k: v for k, v in sorted(b1.bid_win_user_clicks.items(), key=lambda item: item[0])})
print('BWCN - Bid Wins User NO Clicked', {k: v for k, v in sorted(b1.bid_wins_no_click.items(), key=lambda item: item[0])})
print('BLOS - Bids Lost', {k: v for k, v in sorted(b1.bid_lost.items(), key=lambda item: item[0])})


Bidding Rounds 89
BWCY - Bid Wins User Clicked {0: 0, 2: 1, 3: 1, 6: 1}
BWCN - Bid Wins User NO Clicked {0: 3, 1: 2, 5: 1, 8: 1, 9: 3}
BLOS - Bids Lost {0: 9, 1: 12, 2: 5, 3: 7, 4: 9, 5: 5, 6: 5, 7: 9, 8: 21, 9: 6}


In [16]:
b1.bid_360

[['BWCY', 1, 140369076362160, 6, True, 0, 0, True],
 ['BLOS', 2, 140369076362160, 1, False, 0.01, 0.01, None],
 ['BLOS', 3, 140369076362160, 4, False, 0.01263, 0.034125, None],
 ['BLOS',
  4,
  140369076362160,
  5,
  False,
  0.03753750000000001,
  0.03753750000000001,
  None],
 ['BWCY', 5, 140369076362160, 6, True, 0.08493, 0.07898, True],
 ['BLOS', 6, 140369076362160, 9, False, 0.08328, 0.08328, None],
 ['BLOS', 7, 140369076362160, 3, False, 0.00955, 0.09477599999999999, None],
 ['BLOS',
  8,
  140369076362160,
  3,
  False,
  0.05403549999999999,
  0.11373119999999998,
  None],
 ['BLOS', 9, 140369076362160, 9, False, 0.03343, 0.13647743999999998, None],
 ['BLOS',
  10,
  140369076362160,
  6,
  False,
  0.07888103499999997,
  0.13647743999999998,
  None],
 ['BLOS', 11, 140369076362160, 7, False, 0.0823, 0.08351, None],
 ['BLOS',
  12,
  140369076362160,
  9,
  False,
  0.16377292799999996,
  0.16377292799999996,
  None],
 ['BLOS', 13, 140369076362160, 5, False, 0.035025, 0.10744785